In [2]:
import json 
import glob
import itertools
import pandas as pd
import numpy as np
import json

In [16]:
import os

def read_files(base_path):
    file_anagrafica = []
    file_tracking = []
    
    # Cammina attraverso tutte le directory e sottodirectory
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == '.DS_Store':
                continue
            # Controlla se nel nome del file è presente la stringa '_tracking'
            if '_tracking' in file:
                file_tracking.append(os.path.join(root, file))
            else:
                file_anagrafica.append(os.path.join(root, file))
    
    return file_anagrafica, file_tracking

# Percorso della directory principale
base_path = 'SerieA_22-23_tracking'
anagrafica, tracking = read_files(base_path)


In [17]:
anagrafica

['SerieA_22-23_tracking/32/ROMA-MILAN G32.json',
 'SerieA_22-23_tracking/32/NAPOLI-SALERNITANA G32.json',
 'SerieA_22-23_tracking/32/LECCE-UDINESE G32.json',
 'SerieA_22-23_tracking/32/US CREMONESE-HELLAS VERONA G32.json',
 'SerieA_22-23_tracking/32/BOLOGNA-JUVENTUS G32.json',
 'SerieA_22-23_tracking/32/SASSUOLO-EMPOLI G32.json',
 'SerieA_22-23_tracking/32/TORINO-ATALANTA G32.json',
 'SerieA_22-23_tracking/32/INTER-LAZIO G32.json',
 'SerieA_22-23_tracking/32/SPEZIA CALCIO-MONZA G32.json',
 'SerieA_22-23_tracking/32/FIORENTINA-SAMPDORIA G32.json',
 'SerieA_22-23_tracking/35/FIORENTINA-UDINESE G35.json',
 'SerieA_22-23_tracking/35/JUVENTUS-US CREMONESE G35.json',
 'SerieA_22-23_tracking/35/SALERNITANA-ATALANTA G35.json',
 'SerieA_22-23_tracking/35/INTER-SASSUOLO G35.json',
 'SerieA_22-23_tracking/35/HELLAS VERONA-TORINO G35.json',
 'SerieA_22-23_tracking/35/SPEZIA CALCIO-MILAN G35.json',
 'SerieA_22-23_tracking/35/MONZA-NAPOLI G35.json',
 'SerieA_22-23_tracking/35/BOLOGNA-ROMA G35.json',

In [11]:
filename_tracking = tracking[0]
tracks = []
with open(filename_tracking, "r") as file_t:
    for line in file_t:
        tracks.append(json.loads(line))

filename_anagrafica = anagrafica[0]
with open(filename_anagrafica, "r") as file:
    fileData_anagr  = file.read()
    anagrafica = json.loads(fileData_anagr)


def mapSingleTimeFrame(row):

    tracked_row = list(map(lambda d : [d['track_id'],d['x'],d['y'],d['is_visible'],row['period'],
                                       row['timestamp'],row['frame'],row['possession']["group"],
                                       row['possession']["trackable_object"]], row['data']))

    return tracked_row

tracked_rows = list(map(lambda row: mapSingleTimeFrame(row), list(filter(lambda d : d['data'] != [], tracks))))
flatted_tracked_rows = list(itertools.chain.from_iterable(tracked_rows))

home_team_list = [anagrafica['home_team']['id'],anagrafica['home_team']['name']]
away_team_list = [anagrafica['away_team']['id'],anagrafica['away_team']['name']]

anagrafica_team = [home_team_list, away_team_list]

anagrafica_giocatori = list(map(lambda d : [d['team_id'],d['team_player_id'],d['short_name'],d['player_role']['name'],d['trackable_object']], anagrafica['players']))
anagrafica_giocatori_completa = list(map(lambda d : list(d)+[list(filter(lambda c : c[0] == d[0],anagrafica_team))[0][1]], anagrafica_giocatori))


def timehistory_per_player(anagrafica_row):

    ## TROVA IL PLAYER ID ##
    trackable_object = anagrafica_row[-2]
    tracks = list(filter(lambda c : c[0] == trackable_object, flatted_tracked_rows))
    tracks_out = list(map(lambda c : list(c[1:])+anagrafica_row, tracks))

    return tracks_out
    
players_timehistory = list(map(lambda d : timehistory_per_player(d), anagrafica_giocatori_completa))

players_timehistory_f = list(filter(lambda c : len(c) > 0, players_timehistory))

flatten_players_timehistory_f = list(itertools.chain.from_iterable(players_timehistory_f))

def isVisibleToNumeric(is_visible_boolean):
    
    if is_visible_boolean == False:
        new_visible_value = 0.0
    else:
        new_visible_value = 1.0

    return new_visible_value
    
    
def addVisibleValue(timehistory):
    return list(map(lambda row: row[:is_visible_index]+ [isVisibleToNumeric(row[is_visible_index])] + row[1+is_visible_index:], timehistory))


def reduceFrequencyTo2Hz(frame):

    x = float(frame)/10.0
    
    if float(x) - int(x) > 0.25 and float(x) - int(x) < 0.75:
        time_rounded = float(np.round(int(x)+0.5,1))
    elif float(x) - int(x) < 0.25:
        time_rounded =  float(np.round(int(x),0))
    elif float(x) - int(x) > 0.75:
        time_rounded =  float(np.round(int(x)+1,0))

    return time_rounded

pd_tracks = pd.DataFrame(flatten_players_timehistory_f, columns = ['x','y','is_visible','period','timestamp','frame','possess_group','possess_trackable_object','team_id','player_id','player_name','ruolo','track_id','team_name'])
pd_tracks['is_visible_value'] = pd_tracks.apply(lambda row: isVisibleToNumeric(row.is_visible), axis = 1)
pd_tracks['frame_2hz'] = pd_tracks.apply(lambda row: reduceFrequencyTo2Hz(row.frame), axis = 1)

sampled_tracking_data = pd_tracks[['track_id','frame_2hz','period', 'x','y','team_name','player_name','ruolo','possess_group']].groupby(by = ['track_id','frame_2hz','period','team_name','player_name','ruolo','possess_group'], as_index=False).mean()

ball_tracks = list(filter(lambda c : c[0] == 55, flatted_tracked_rows))
ball_tracks = pd.DataFrame(ball_tracks, columns = ['track_id','x','y','is_visible','period','timestamp','frame','possess_group','possess_trackable_object'])
ball_tracks['frame_2hz'] = ball_tracks.apply(lambda row: reduceFrequencyTo2Hz(row.frame), axis = 1)
sampled_ball_data = ball_tracks[['track_id','frame_2hz','x','y','possess_group']].groupby(by = ['track_id','frame_2hz','possess_group'], as_index=False).mean()

tracking_data = pd.merge(sampled_tracking_data, sampled_ball_data[['frame_2hz', 'x', 'y']], on = ['frame_2hz'], how = 'left')
tracking_data.columns = ['track_id', 'frame_2hz', 'period', 'team_name', 'player_name', 'ruolo', 'possess_group', 'x', 'y', 'x_ball', 'y_ball']


# Determina il lato di attacco basato sul periodo e se il team_name corrisponde al team di casa
def determine_attack_side(row):
    if row['team_name'] == anagrafica['home_team']['name']:
        # La squadra del giocatore è la squadra di casa
        return anagrafica['home_team_side'][0] if row['period'] == 1 else anagrafica['home_team_side'][1]
    else:
        # La squadra del giocatore è la squadra in trasferta, quindi l'attacco è invertito
        return anagrafica['home_team_side'][1] if row['period'] == 1 else anagrafica['home_team_side'][0]

tracking_data['attack_side'] = tracking_data.apply(determine_attack_side, axis=1)

# Funzione per calcolare la distanza euclidea
def calc_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Filtriamo i portieri
tracking_data = tracking_data[tracking_data['ruolo'] != 'Goalkeeper']

# Identifica chi sono i difensori in base alla direzione di attacco e alla posizione della palla
def is_defender(row, home_team, away_team):
    if row['possess_group'] == 'home team':
        defending_team = away_team
    else:
        defending_team = home_team

    if row['team_name'] == defending_team:
        if row['attack_side'] == 'left_to_right':
            return row['x'] <= row['x_ball']
        else:
            return row['x'] >= row['x_ball']
    return False


home_team = anagrafica['home_team']['name']
away_team = anagrafica['away_team']['name']
tracking_data['is_defender'] = tracking_data.apply(is_defender, args=(home_team, away_team), axis=1)

# Funzione per calcolare il difensore più vicino per ogni frame
def analyze_frame(frame_data):
    defenders = frame_data[frame_data['is_defender']]
    attackers = frame_data[~frame_data['is_defender']]
    
    results = []
    for _, defender in defenders.iterrows():
        min_distance = np.inf
        closest_attacker = None
        for _, attacker in attackers.iterrows():
            distance = calc_distance(defender['x'], defender['y'], attacker['x'], attacker['y'])
            if distance < min_distance:
                min_distance = distance
                closest_attacker = attacker['player_name']
        
        results.append({
            'frame': defender['frame_2hz'],
            'defender': defender['player_name'],
            'closest_attacker': closest_attacker,
            'min_distance': min_distance,
            'possession_group': defender['possess_group']
        })
    return results

# Applichiamo la funzione ad ogni frame
frame_results = []
grouped_frames = tracking_data.groupby('frame_2hz')
for frame, frame_data in grouped_frames:
    frame_results.extend(analyze_frame(frame_data))

# Creiamo un DataFrame con i risultati
results_df = pd.DataFrame(frame_results)

results_df = results_df[results_df['defender'] != results_df['closest_attacker']]

results_df['possession_change'] = (results_df['possession_group'] != results_df['possession_group'].shift(1)).astype(int)

results_df['possession_id'] = results_df['possession_change'].cumsum()

possession_durations = results_df.groupby('possession_id').agg(
    start_frame=('frame', 'min'),
    end_frame=('frame', 'max')
).reset_index()
possession_durations['duration'] = (possession_durations['end_frame'] - possession_durations['start_frame'])

# Filtra per mantenere solo i gruppi di possesso che durano almeno 5 secondi
valid_possessions = possession_durations[possession_durations['duration'] >= 5]['possession_id']

# Utilizza solo i dati dei gruppi di possesso validi
valid_groups = results_df[results_df['possession_id'].isin(valid_possessions)]

# Aggiunta del tracking dei cambi di marcatura
valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']
changes_per_defender = valid_groups.groupby(['defender', 'possession_id'])['change_in_marking'].sum().groupby('defender').mean()

# Calcolo della distanza media
mean_distances = valid_groups.groupby('defender')['min_distance'].mean()
# Calcola la distanza media per ogni attaccante e ogni possesso
attacker_distances = valid_groups.groupby(['possession_id', 'closest_attacker'])['min_distance'].mean().reset_index()

# Trova l'attaccante più e meno marcato globalmente
# Calcola prima la distanza media generale per ogni attaccante attraverso tutti i possedimenti
overall_distances = attacker_distances.groupby('closest_attacker')['min_distance'].mean()

most_marked_attacker = overall_distances.idxmin()
least_marked_attacker = overall_distances.idxmax()

# Recupera la distanza media per il più e meno marcato
most_marked_attacker_distance = overall_distances.min()
least_marked_attacker_distance = overall_distances.max()

In [13]:
# Aggiunta del tracking dei cambi di marcatura
valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']
changes_per_defender = valid_groups.groupby(['defender', 'possession_id'])['change_in_marking'].sum().groupby('defender').mean()

# Calcolo della distanza media
mean_distances = valid_groups.groupby('defender')['min_distance'].mean()
# Calcola la distanza media per ogni attaccante e ogni possesso
attacker_distances = results_df.groupby(['possession_id', 'closest_attacker'])['min_distance'].mean().reset_index()

# Trova l'attaccante più e meno marcato globalmente
# Calcola prima la distanza media generale per ogni attaccante attraverso tutti i possedimenti
overall_distances = attacker_distances.groupby('closest_attacker')['min_distance'].mean()

most_marked_attacker = overall_distances.idxmin()
least_marked_attacker = overall_distances.idxmax()

# Recupera la distanza media per il più e meno marcato
most_marked_attacker_distance = overall_distances.min()
least_marked_attacker_distance = overall_distances.max()

# Stampare i risultati generali
print("Media dei cambi di marcatura per difensore:\n", changes_per_defender)
print("Distanza media per difensore:\n", mean_distances)
print("Attaccante più marcato globalmente: {}, Distanza media: {:.2f}".format(most_marked_attacker, most_marked_attacker_distance))
print("Attaccante con più libertà globalmente: {}, Distanza media: {:.2f}".format(least_marked_attacker, least_marked_attacker_distance))

Media dei cambi di marcatura per difensore:
 defender
A. Belotti         5.708333
A. Saelemaekers    2.272727
B. Cristante       4.315789
Brahim Diaz        3.250000
C. De Ketelaere    2.823529
D. Calabria        2.971014
D. Origi           2.750000
E. Bove            5.000000
F. Tomori          3.250000
G. Mancini         4.519481
I. Bennacer        2.977273
Ibañez             4.285714
L. Pellegrini      4.797297
L. Spinazzola      4.081081
M. Camara          6.000000
M. Kumbulla        2.555556
M. Thiaw           2.729730
N. Matić           4.657895
O. Giroud          2.225000
O. Solbakken       4.500000
P. Kalulu          3.166667
R. Krunić          3.161765
Rafael Leão        1.921569
S. El Shaarawy     3.909091
S. Kjær            3.039216
S. Tonali          3.373134
T. Abraham         3.683333
T. Hernández       3.223881
Z. Çelik           3.447368
Name: change_in_marking, dtype: float64
Distanza media per difensore:
 defender
A. Belotti         6.745711
A. Saelemaekers    5.31644

/var/folders/n2/9_7g8plj0bl7kt3pzlkmfdm80000gn/T/ipykernel_32292/1122480269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']


In [1]:
import json
import os
import numpy as np
import pandas as pd
import itertools

def load_json_data(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

def process_game_data(tracking_file, anagrafica_file):
    # Importa i dati
    with open(tracking_file, 'r') as file:
        tracks = [json.loads(line) for line in file]
    
    anagrafica = load_json_data(anagrafica_file)
    
    def mapSingleTimeFrame(row):

        tracked_row = list(map(lambda d : [d['track_id'],d['x'],d['y'],d['is_visible'],row['period'],
                                        row['timestamp'],row['frame'],row['possession']["group"],
                                        row['possession']["trackable_object"]], row['data']))

        return tracked_row

    tracked_rows = list(map(lambda row: mapSingleTimeFrame(row), list(filter(lambda d : d['data'] != [], tracks))))
    flatted_tracked_rows = list(itertools.chain.from_iterable(tracked_rows))

    home_team_list = [anagrafica['home_team']['id'],anagrafica['home_team']['name']]
    away_team_list = [anagrafica['away_team']['id'],anagrafica['away_team']['name']]

    anagrafica_team = [home_team_list, away_team_list]

    anagrafica_giocatori = list(map(lambda d : [d['team_id'],d['team_player_id'],d['short_name'],d['player_role']['name'],d['trackable_object']], anagrafica['players']))
    anagrafica_giocatori_completa = list(map(lambda d : list(d)+[list(filter(lambda c : c[0] == d[0],anagrafica_team))[0][1]], anagrafica_giocatori))


    def timehistory_per_player(anagrafica_row):

        ## TROVA IL PLAYER ID ##
        trackable_object = anagrafica_row[-2]
        tracks = list(filter(lambda c : c[0] == trackable_object, flatted_tracked_rows))
        tracks_out = list(map(lambda c : list(c[1:])+anagrafica_row, tracks))

        return tracks_out
        
    players_timehistory = list(map(lambda d : timehistory_per_player(d), anagrafica_giocatori_completa))

    players_timehistory_f = list(filter(lambda c : len(c) > 0, players_timehistory))

    flatten_players_timehistory_f = list(itertools.chain.from_iterable(players_timehistory_f))

    def isVisibleToNumeric(is_visible_boolean):
        
        if is_visible_boolean == False:
            new_visible_value = 0.0
        else:
            new_visible_value = 1.0

        return new_visible_value
        
        
    def addVisibleValue(timehistory):
        return list(map(lambda row: row[:is_visible_index]+ [isVisibleToNumeric(row[is_visible_index])] + row[1+is_visible_index:], timehistory))


    def reduceFrequencyTo2Hz(frame):

        x = float(frame)/10.0
        
        if float(x) - int(x) > 0.25 and float(x) - int(x) < 0.75:
            time_rounded = float(np.round(int(x)+0.5,1))
        elif float(x) - int(x) < 0.25:
            time_rounded =  float(np.round(int(x),0))
        elif float(x) - int(x) > 0.75:
            time_rounded =  float(np.round(int(x)+1,0))

        return time_rounded

    pd_tracks = pd.DataFrame(flatten_players_timehistory_f, columns = ['x','y','is_visible','period','timestamp','frame','possess_group','possess_trackable_object','team_id','player_id','player_name','ruolo','track_id','team_name'])
    pd_tracks['is_visible_value'] = pd_tracks.apply(lambda row: isVisibleToNumeric(row.is_visible), axis = 1)
    pd_tracks['frame_2hz'] = pd_tracks.apply(lambda row: reduceFrequencyTo2Hz(row.frame), axis = 1)

    sampled_tracking_data = pd_tracks[['track_id','frame_2hz','period', 'x','y','team_name','player_name','ruolo','possess_group']].groupby(by = ['track_id','frame_2hz','period','team_name','player_name','ruolo','possess_group'], as_index=False).mean()

    ball_tracks = list(filter(lambda c : c[0] == 55, flatted_tracked_rows))
    ball_tracks = pd.DataFrame(ball_tracks, columns = ['track_id','x','y','is_visible','period','timestamp','frame','possess_group','possess_trackable_object'])
    ball_tracks['frame_2hz'] = ball_tracks.apply(lambda row: reduceFrequencyTo2Hz(row.frame), axis = 1)
    sampled_ball_data = ball_tracks[['track_id','frame_2hz','x','y','possess_group']].groupby(by = ['track_id','frame_2hz','possess_group'], as_index=False).mean()

    tracking_data = pd.merge(sampled_tracking_data, sampled_ball_data[['frame_2hz', 'x', 'y']], on = ['frame_2hz'], how = 'left')
    tracking_data.columns = ['track_id', 'frame_2hz', 'period', 'team_name', 'player_name', 'ruolo', 'possess_group', 'x', 'y', 'x_ball', 'y_ball']


    # Determina il lato di attacco basato sul periodo e se il team_name corrisponde al team di casa
    def determine_attack_side(row):
        if row['team_name'] == anagrafica['home_team']['name']:
            # La squadra del giocatore è la squadra di casa
            return anagrafica['home_team_side'][0] if row['period'] == 1 else anagrafica['home_team_side'][1]
        else:
            # La squadra del giocatore è la squadra in trasferta, quindi l'attacco è invertito
            return anagrafica['home_team_side'][1] if row['period'] == 1 else anagrafica['home_team_side'][0]

    tracking_data['attack_side'] = tracking_data.apply(determine_attack_side, axis=1)

    # Funzione per calcolare la distanza euclidea
    def calc_distance(x1, y1, x2, y2):
        return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    # Filtriamo i portieri
    tracking_data = tracking_data[tracking_data['ruolo'] != 'Goalkeeper']

    # Identifica chi sono i difensori in base alla direzione di attacco e alla posizione della palla
    def is_defender(row, home_team, away_team):
        if row['possess_group'] == 'home team':
            defending_team = away_team
        else:
            defending_team = home_team

        if row['team_name'] == defending_team:
            if row['attack_side'] == 'left_to_right':
                return row['x'] <= row['x_ball']
            else:
                return row['x'] >= row['x_ball']
        return False


    home_team = anagrafica['home_team']['name']
    away_team = anagrafica['away_team']['name']
    tracking_data['is_defender'] = tracking_data.apply(is_defender, args=(home_team, away_team), axis=1)

    # Funzione per calcolare il difensore più vicino per ogni frame
    def analyze_frame(frame_data):
        defenders = frame_data[frame_data['is_defender']]
        attackers = frame_data[~frame_data['is_defender']]
        
        results = []
        for _, defender in defenders.iterrows():
            min_distance = np.inf
            closest_attacker = None
            for _, attacker in attackers.iterrows():
                distance = calc_distance(defender['x'], defender['y'], attacker['x'], attacker['y'])
                if distance < min_distance:
                    min_distance = distance
                    closest_attacker = attacker['player_name']
            
            results.append({
                'frame': defender['frame_2hz'],
                'defender': defender['player_name'],
                'closest_attacker': closest_attacker,
                'min_distance': min_distance,
                'possession_group': defender['possess_group']
            })
        return results

    # Applichiamo la funzione ad ogni frame
    frame_results = []
    grouped_frames = tracking_data.groupby('frame_2hz')
    for frame, frame_data in grouped_frames:
        frame_results.extend(analyze_frame(frame_data))

    # Creiamo un DataFrame con i risultati
    results_df = pd.DataFrame(frame_results)

    results_df = results_df[results_df['defender'] != results_df['closest_attacker']]

    results_df['possession_change'] = (results_df['possession_group'] != results_df['possession_group'].shift(1)).astype(int)

    results_df['possession_id'] = results_df['possession_change'].cumsum()

    possession_durations = results_df.groupby('possession_id').agg(
        start_frame=('frame', 'min'),
        end_frame=('frame', 'max')
    ).reset_index()
    possession_durations['duration'] = (possession_durations['end_frame'] - possession_durations['start_frame'])

    # Filtra per mantenere solo i gruppi di possesso che durano almeno 5 secondi
    valid_possessions = possession_durations[possession_durations['duration'] >= 5]['possession_id']

    # Utilizza solo i dati dei gruppi di possesso validi
    valid_groups = results_df[results_df['possession_id'].isin(valid_possessions)]


    # Aggiunta del tracking dei cambi di marcatura
    valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']
    changes_per_defender = valid_groups.groupby(['defender', 'possession_id'])['change_in_marking'].sum().groupby('defender').mean()

    # Calcolo della distanza media
    mean_distances = valid_groups.groupby('defender')['min_distance'].mean()
    
    # Calcola la distanza media per ogni attaccante e ogni possesso
    attacker_distances = valid_groups.groupby(['possession_id', 'closest_attacker'])['min_distance'].mean().reset_index()
    overall_distances = attacker_distances.groupby('closest_attacker')['min_distance'].mean()

    most_marked_attacker = overall_distances.idxmin()
    least_marked_attacker = overall_distances.idxmax()
    most_marked_attacker_distance = overall_distances.min()
    least_marked_attacker_distance = overall_distances.max()

    # Organizza i dati finali in un dizionario
    results = {
        'match_info': {
            'home_team': anagrafica['home_team']['name'],
            'away_team': anagrafica['away_team']['name'],
            'matchday': tracking_file.split('/')[-2]  # Estrae il numero della giornata dal percorso del file
        },
        'statistics': {
            'changes_per_defender': changes_per_defender.to_dict(),
            'mean_distances_per_defender': mean_distances.to_dict(),
            'most_marked_attacker': most_marked_attacker,
            'least_marked_attacker': least_marked_attacker,
            'most_marked_attacker_distance': most_marked_attacker_distance,
            'least_marked_attacker_distance': least_marked_attacker_distance
        }
    }

    return results

def process_all_games(directory):
    all_games_results = []
    for root, dirs, files in os.walk(directory):
        
        tracking_files = [f for f in files if 'tracking.json' in f and f != '.DS_Store']
        anagrafica_files = [f for f in files if 'tracking.json' not in f and f != '.DS_Store']
        
        for tracking_file, anagrafica_file in zip(sorted(tracking_files), sorted(anagrafica_files)):
            tracking_path = os.path.join(root, tracking_file)
            anagrafica_path = os.path.join(root, anagrafica_file)
            print((tracking_path, anagrafica_path))
            game_results = process_game_data(tracking_path, anagrafica_path)
            all_games_results.append(game_results)
    
    # Salvare i risultati aggregati in un JSON
    with open('all_games_results.json', 'w') as json_file:
        json.dump(all_games_results, json_file)

# Processa tutte le partite nella directory specificata
process_all_games('SerieA_22-23_tracking')




('SerieA_22-23_tracking/32/BOLOGNA-JUVENTUS G32_tracking.json', 'SerieA_22-23_tracking/32/BOLOGNA-JUVENTUS G32.json')


/var/folders/n2/9_7g8plj0bl7kt3pzlkmfdm80000gn/T/ipykernel_32508/2964926902.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']


('SerieA_22-23_tracking/32/FIORENTINA-SAMPDORIA G32_tracking.json', 'SerieA_22-23_tracking/32/FIORENTINA-SAMPDORIA G32.json')


/var/folders/n2/9_7g8plj0bl7kt3pzlkmfdm80000gn/T/ipykernel_32508/2964926902.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']


('SerieA_22-23_tracking/32/INTER-LAZIO G32_tracking.json', 'SerieA_22-23_tracking/32/INTER-LAZIO G32.json')


...

/var/folders/n2/9_7g8plj0bl7kt3pzlkmfdm80000gn/T/ipykernel_32508/2964926902.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']


('SerieA_22-23_tracking/25/SPEZIA CALCIO-HELLAS VERONA G25_tracking.json', 'SerieA_22-23_tracking/25/SPEZIA CALCIO-HELLAS VERONA G25.json')


/var/folders/n2/9_7g8plj0bl7kt3pzlkmfdm80000gn/T/ipykernel_32508/2964926902.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']


('SerieA_22-23_tracking/25/TORINO-BOLOGNA G25_tracking.json', 'SerieA_22-23_tracking/25/TORINO-BOLOGNA G25.json')


/var/folders/n2/9_7g8plj0bl7kt3pzlkmfdm80000gn/T/ipykernel_32508/2964926902.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_groups['change_in_marking'] = valid_groups.groupby(['defender', 'possession_id'])['closest_attacker'].shift() != valid_groups['closest_attacker']
